In [1]:
import os
import numpy as np
import pandas as pd
import constants

from collections import defaultdict
from sklearn.model_selection import KFold

In [2]:
class_to_label = {'ND':0, 'D':1}
labels_df = pd.read_csv(constants.LABEL_FILE)

In [14]:
class_to_slide = {}
class_to_slide['D'] = list(labels_df['Slide'][labels_df['Nucleoli']  == 'D'])
class_to_slide['ND'] = list(labels_df['Slide'][labels_df['Nucleoli']  == 'ND'])
class_to_slide

{'D': ['FLT1',
  'FLT2',
  'FLT3',
  'FLT4',
  'FLT5',
  'FLT7',
  'FLT8',
  'FLT9',
  'FLT10',
  'FLT12',
  'FLT13',
  'FLT15',
  'FLN7'],
 'ND': ['FLT6', 'FLT14', 'FLN1', 'FLN2', 'FLN3', 'FLN4', 'FLN5', 'FLN6']}

In [12]:
slide_to_class = 

array(['FLT1', 'FLT2', 'FLT3', 'FLT4', 'FLT5', 'FLT7', 'FLT8', 'FLT9',
       'FLT10', 'FLT12', 'FLT13', 'FLT15', 'FLN7'], dtype=object)